In [2]:
# Basic functions created
# select_random_words = selects random words
# create_poems = merge 4 words to string to get poem like structure
# store_syllables = saves the sum of syllables in each verse
# Grammar_checker = Check grammar

In [3]:
# pip install --upgrade pip


In [4]:
# pip install language-tool-python
# !pip install grammar-check

In [167]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
# import nltk 
# import language_check

In [6]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i)
words_df = temp 

In [7]:
# generate random 16 numbers
def select_random_words(words_df):
    words_array1 = []
    while len(words_array1) <= 15:
        random_number = random.randint(0, len(words_df)-1)
        if words_df[random_number] != 0:
            words_array1.append(words_df[random_number])
    # print(len(words_array1))
    return words_array1

In [8]:
def create_poems(arr):
  j = 0
  poem = [0 for _ in range(4)]
  for i in range(1,5):
    temp =  arr[4*(i-1):4*i]
    poem[i-1] = temp
    poem[i-1]=' '.join(poem[i-1])
    print(poem[i-1])
  print("=========================================================")
  return poem

In [9]:
def store_syllables(arr):
    a = [0 for _ in range(4)]
    temp = 0
    j = 0
    for words,i in zip(arr,range(0,16)):
        temp = temp + syllables.estimate(words)
        if i in [3,7,11,15]:
            a[j] = temp
            # print(a[j])
            temp = 0
            j += 1
    return a

In [10]:
def Grammar_checker(text):
    error = 0
    poem_error = []
    tool = language_tool_python.LanguageTool('en-US')
    for lines in text:
        matches = tool.check(lines)
        if len(matches)>0:
            error = error + matches[0].errorLength
    poem_error.append(error)
    return poem_error

In [11]:
# words_array = select_random_words()
# poem = create_poems(words_array)
# sum_syllables = store_syllables(words_array)

In [18]:
# 1. create_set_of_poems(no of poems to be made)
# 	1. select_random_words()
# 	2. create_poems(from random words)
# 	3. store_syllables(cal sum syllables count)
# 	4. Grammar_checker(poems created in 2.)
# 	5. poem_population(append all poems)
# 	   syllables_count(append all syllablessum)
# 	6. grammar_error(append all grammar error 4.)
# 	7. words_array_population(append all words)
# 	8. DataFrame of all population

def create_set_of_poems(n):
    poem_population = []
    syllables_count = []
    grammar_error = []
    words_array_population = []
    for i in range(n):
        words_array = select_random_words(words_df)
        poem = create_poems(words_array)
        sum_syllables = store_syllables(words_array)
        poem_error = Grammar_checker(poem)
        poem_population.append(poem)
        syllables_count.append(sum_syllables)
        grammar_error.append(poem_error)
        words_array_population.append(words_array)
    words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
    Final = np.hstack((poem_population,syllables_count,grammar_error))
    Pop_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
    Pop_data = pd.concat([Pop_data,words_array2],axis=1)
    return words_array_population,Pop_data




In [145]:
def crossover(p1,p2,cross):
    part1 = []
    part2 = []
    part3 = []
    child1 = []
    child2 = []
    j = 0
    k = 0
    random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
    random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
    for i in random_number1:
        part1.append(p1[j:i])
        j = i
    for m in random_number2:
        part2.append(p2[k:m])
        k = m
    part1.append(p1[j:])
    part2.append(p2[k:])
    part2.extend(part1)
    # print(part2)
    random.shuffle(part2)
    # print(part2)
    for i in range(len(part2)):
        part3.extend(part2[i])
    for i in range(int(len(part3)/2)):
        child1.append(part3[i])
    for i in range(int(len(part3)/2),int(len(part3))):
        child2.append(part3[i])
    if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
        child1,child2,child1_poem,child2_poem = crossover(child1,child2,cross)
    child1_poem = create_poems(child1)
    child2_poem = create_poems(child2)
    return child1,child2,child1_poem,child2_poem

In [14]:
def child_poems(words_array,cross):
    child_population = []
    syllables_count = []
    grammer_error = []
    words_array_child = []
    for i in range(0,len(words_array)-1,2):
        child1,child2,child1_poem,child2_poem = crossover(words_array[i],words_array[i+1],cross)
        child_population.append(child1_poem)
        child_population.append(child2_poem)
        sum_syllables1 = store_syllables(child1)
        sum_syllables2 = store_syllables(child2)
        poem_error1 = Grammar_checker(child1_poem)
        poem_error2 = Grammar_checker(child2_poem)
        syllables_count.append(sum_syllables1)
        syllables_count.append(sum_syllables2)
        grammer_error.append(poem_error1)
        grammer_error.append(poem_error2)
        words_array_child.append(child1)
        words_array_child.append(child2)
    words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
    Final1 = np.hstack((child_population,syllables_count,grammer_error))
    child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
    child_data = pd.concat([child_data,words_array_child],axis=1)
    return words_array_child,child_data


In [131]:
def tournament(Population,m):
    p = int(m/3)
    n = m-p
    df = pd.DataFrame()
    # print(Population.head())
    selected_polulation = pd.DataFrame()
    Population.sort_values(by=['GrammarError'], inplace=True)
    selected_polulation = Population.iloc[:n]
    # selected_polulation = Population.loc[(Population["Syllables1"] == 5)&(Population["Syllables2"] == 6)&(Population["Syllables3"] == 6)&(Population["Syllables4"] == 5)]
    # selected_polulation = selected_polulation.iloc[:n]
    random_number = [random.randint(n, len(Population)-1) for i in range(p)]
    for i in random_number:
        df = df.append(Population.iloc[i])
    selected_polulation = pd.concat([selected_polulation,df],axis=0)
    return selected_polulation,selected_polulation["words_array"]

    

In [19]:
n = 5
cross = 2

words_array,Population_data = create_set_of_poems(n)


Of great begin Her
trembling crabs frowns billows
mens saying call his
What to stone thee
Lord Southlands lifes task
This built berries is
is upto the the
Ay as of and
To gabies delight night
LOOKING and born pride
looks since the smell
Cabinetsbe Lovers ingredients Force
wreck things Door about
south Like that for
clay to meddleone the
haycocks your how a
slips at he But
judge waters withholden sorrowfully
wellcurb eyes some little
never in the weep


In [138]:
Selection_number = 4
Selected_Population_data,selected_polulation_word_array = tournament(Population_data,Selection_number)

In [140]:
Selected_Population_data.head()

,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,Of great begin Her,trembling crabs frowns billows,mens saying call his,What to stone thee,5,6,4,5,13,"[Of, great, begin, Her, trembling, crabs, frow..."
1,Lord Southlands lifes task,This built berries is,is upto the the,Ay as of and,6,5,5,4,14,"[Lord, Southlands, lifes, task, This, built, b..."
2,To gabies delight night,LOOKING and born pride,looks since the smell,Cabinetsbe Lovers ingredients Force,6,5,5,11,21,"[To, gabies, delight, night, LOOKING, and, bor..."
3,wreck things Door about,south Like that for,clay to meddleone the,haycocks your how a,5,4,6,5,22,"[wreck, things, Door, about, south, Like, that..."


In [ ]:

words_array_child,child_population = child_poems(selected_polulation_word_array,cross)

In [147]:
child_population

,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,begin Her trembling crabs,frowns billows mens saying,call his What to,stone thee the the,6,5,4,5,20,"[begin, Her, trembling, crabs, frowns, billows..."
1,Ay as of and,upto Lord Southlands lifes,task This built berries,is is Of great,4,7,5,4,12,"[Ay, as, of, and, upto, Lord, Southlands, life..."
2,pride looks since To,gabies delight the smell,Cabinetsbe Lovers haycocks your,how a ingredients Force,6,6,9,7,24,"[pride, looks, since, To, gabies, delight, the..."
3,wreck things Door about,south Like that for,clay to meddleone the,night LOOKING and born,5,4,6,4,19,"[wreck, things, Door, about, south, Like, that..."


In [178]:
# class poem_generation: 
#     pd.set_option('display.max_rows', None)
#     pd.set_option('display.max_columns', None)
#     pd.set_option('display.width', None)
#     pd.set_option('display.max_colwidth', None)
#     def create_set_of_poems(self,n):
#             poem_population = []
#             syllables_count = []
#             grammar_error = []
#             words_array_population = []
#             for i in range(n):
#                 words_array = select_random_words(words_df)
#                 poem = create_poems(words_array)
#                 sum_syllables = store_syllables(words_array)
#                 poem_error = Grammar_checker(poem)
#                 poem_population.append(poem)
#                 syllables_count.append(sum_syllables)
#                 grammar_error.append(poem_error)
#                 words_array_population.append(words_array)
#             words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
#             Final = np.hstack((poem_population,syllables_count,grammar_error))
#             Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
#             Pop_data = pd.concat([Population_data,words_array2],axis=1)
#             print("PARENTS")
#             display(Pop_data)
#             return words_array_population,Pop_data

#     def child_poems(self,words_array,cross):
#         child_population = []
#         syllables_count = []
#         grammer_error = []
#         words_array_child = []
#         for i in range(0,len(words_array)-1,2):
#             child1,child2,child1_poem,child2_poem = crossover(words_array[i],words_array[i+1],cross)
#             child_population.append(child1_poem)
#             child_population.append(child2_poem)
#             sum_syllables1 = store_syllables(child1)
#             sum_syllables2 = store_syllables(child2)
#             poem_error1 = Grammar_checker(child1_poem)
#             poem_error2 = Grammar_checker(child2_poem)
#             syllables_count.append(sum_syllables1)
#             syllables_count.append(sum_syllables2)
#             grammer_error.append(poem_error1)
#             grammer_error.append(poem_error2)
#             words_array_child.append(child1)
#             words_array_child.append(child2)
#         words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
#         Final1 = np.hstack((child_population,syllables_count,grammer_error))
#         child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
#         child_data = pd.concat([child_data,words_array_child],axis=1)
#         print("CHILD")
#         display(child_data)
#         return words_array_child,child_data

#     def select_random_words(self,words_df):
#         words_array1 = []
#         while len(words_array1) <= 15:
#             random_number = random.randint(0, len(words_df)-1)
#             if words_df[random_number] != 0:
#                 words_array1.append(words_df[random_number])
#         # print(len(words_array1))
#         return words_array1

#     def create_poems(self,arr):
        
#         poem = [0 for _ in range(4)]
#         for i in range(1,5):
#             temp =  arr[4*(i-1):4*i]
#             poem[i-1] = temp
#             poem[i-1]=' '.join(poem[i-1])
#             print(poem[i-1])
#         print("=========================================================")
#         return poem

#     def store_syllables(self,arr):
#         a = [0 for _ in range(4)]
#         temp = 0
#         j = 0
#         for words,i in zip(arr,range(0,16)):
#             temp = temp + syllables.estimate(words)
#             if i in [3,7,11,15]:
#                 a[j] = temp
#                 # print(a[j])
#                 temp = 0
#                 j += 1
#         return a

#     def Grammar_checker(self,text):
#         error = 0
#         poem_error = []
#         tool = language_tool_python.LanguageTool('en-US')
#         for lines in text:
#             matches = tool.check(lines)
#             if len(matches)>0:
#                 error = error + matches[0].errorLength
#         poem_error.append(error)
#         return poem_error

#     def crossover(self,p1,p2,cross):
#         part1 = []
#         part2 = []
#         part3 = []
#         child1 = []
#         child2 = []
#         j = 0
#         k = 0
#         random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
#         random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
#         for i in random_number1:
#             part1.append(p1[j:i])
#             j = i
#         for i in random_number2:
#             part2.append(p2[k:i])
#             k = i
#         part1.append(p1[j:])
#         part2.append(p2[k:])
#         part2.extend(part1)
#         random.shuffle(part2)
#         for i in range(len(part2)):
#             part3.extend(part2[i])
#         for i in range(int(len(part3)/2)):
#             child1.append(part3[i])
#         for i in range(int(len(part3)/2),int(len(part3))):
#             child2.append(part3[i])
#         if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
#             child1, child2,child1_poem,child2_poem = crossover(child1,child2,cross)
#         child1_poem = create_poems(child1)
#         child2_poem = create_poems(child2)
#         return child1,child2,child1_poem,child2_poem

#     def tournament(self,Population,m):
#         p = int(m/3)
#         n = m-p
#         df = pd.DataFrame()
#         # print(Population.head())
#         selected_polulation = pd.DataFrame()
#         Population.sort_values(by=['GrammarError'], inplace=True)
#         selected_polulation = Population.iloc[:n]
#         # selected_polulation = Population.loc[(Population["Syllables1"] == 5)&(Population["Syllables2"] == 6)&(Population["Syllables3"] == 6)&(Population["Syllables4"] == 5)]
#         # selected_polulation = selected_polulation.iloc[:n]
#         random_number = [random.sample(n, len(Population)-1) for i in range(p)]
#         for i in random_number:
#             df.append(Population.iloc[i])
#         selected_polulation = pd.concat([selected_polulation,df],axis=0)
#         print("SELECTED")
#         display(selected_polulation)
#         return selected_polulation,selected_polulation["words_array"]       




#     def __init__(self,words_df,n,cross,Selection_number):
#         assert n>1 ,f"Population {n} cannot be less than or equal to 1"
#         assert n>=Selection_number, f"Cannot select {Selection_number} poems from total pololation of {n}"
#         assert cross>1, f"Minimum number for crossover should be 1"
#         words_array,Population_data = self.create_set_of_poems(n)
#         Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number)
#         words_array_child,child_population = self.child_poems(selected_polulation_word_array,cross)




In [179]:
# poem = poem_generation(words_df,2,2,2)

she and I that
thee ancient my Then
ticket dawn came elate
Yet enter Of to
Bore jovial Watching hoof
fall they go be
the blows pages babes
click it in prince
PARENTS


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,she and I that,thee ancient my Then,ticket dawn came elate,Yet enter Of to,4,5,7,5,13,"[she, and, I, that, thee, ancient, my, Then, ticket, dawn, came, elate, Yet, enter, Of, to]"
1,Bore jovial Watching hoof,fall they go be,the blows pages babes,click it in prince,6,4,6,5,12,"[Bore, jovial, Watching, hoof, fall, they, go, be, the, blows, pages, babes, click, it, in, prince]"


SELECTED


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
1,Bore jovial Watching hoof,fall they go be,the blows pages babes,click it in prince,6,4,6,5,12,"[Bore, jovial, Watching, hoof, fall, they, go, be, the, blows, pages, babes, click, it, in, prince]"
0,she and I that,thee ancient my Then,ticket dawn came elate,Yet enter Of to,4,5,7,5,13,"[she, and, I, that, thee, ancient, my, Then, ticket, dawn, came, elate, Yet, enter, Of, to]"


[['Bore', 'jovial', 'Watching', 'hoof', 'fall', 'they'], ['go', 'be', 'the', 'blows', 'pages', 'babes', 'click', 'it'], ['in', 'prince'], ['she', 'and', 'I'], ['that', 'thee', 'ancient', 'my', 'Then', 'ticket', 'dawn', 'came', 'elate', 'Yet', 'enter', 'Of'], ['to']]
[['that', 'thee', 'ancient', 'my', 'Then', 'ticket', 'dawn', 'came', 'elate', 'Yet', 'enter', 'Of'], ['go', 'be', 'the', 'blows', 'pages', 'babes', 'click', 'it'], ['in', 'prince'], ['Bore', 'jovial', 'Watching', 'hoof', 'fall', 'they'], ['she', 'and', 'I'], ['to']]
that thee ancient my
Then ticket dawn came
elate Yet enter Of
go be the blows
pages babes click it
in prince Bore jovial
Watching hoof fall they
she and I to
CHILD


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,that thee ancient my,Then ticket dawn came,elate Yet enter Of,go be the blows,5,5,7,4,11,"[that, thee, ancient, my, Then, ticket, dawn, came, elate, Yet, enter, Of, go, be, the, blows]"
1,pages babes click it,in prince Bore jovial,Watching hoof fall they,she and I to,6,6,5,4,10,"[pages, babes, click, it, in, prince, Bore, jovial, Watching, hoof, fall, they, she, and, I, to]"
